In [55]:
import requests
import json
import sys
import pandas as pd
import csv
from  keys  import  client_id, api_key

In [68]:
# FOR TOMORROW
"""
We need to fix the final reviews_to_csv function at the bottom. It is not working with lists of 1000 businesses.

See for LA:
term: Film Production

location: Los Angeles

Categories: Video/Film Production, Recording & Rehearsal Studios, Advertising, Studio Taping

"""

'\nWe need to fix the final reviews_to_csv function at the bottom. It is not working with lists of 1000 businesses.\n\nSee for LA:\nterm: Film Production\n\nlocation: Los Angeles\n\nCategories: Video/Film Production, Recording & Rehearsal Studios, Advertising, Studio Taping\n\n'

In [69]:
'''
 This will save the URL for the Yelp API and the API key.  
 Running this cell prompts the user to input variables that will be used to search the Yelp API
'''
url =  'https://api.yelp.com/v3/businesses/search' #points to url of user's yelp developer page
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
term = input('What sort of business do you think you can do? (plumbers, takeout, etc.): ')
location = input('Near where?: ')
categories = input('What sort of industry would you like to disrupt? You may list multiple categories, separated with a comma: ')
spec = f'{term}_{location}_{categories}_data'
# Note on categories, they appear to be optional. You may sepparate entries with a comma to search multiple categories, search 'All' for all, or leave the field blank.
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
                "offset": 0
            }

What sort of business do you think you can do? (plumbers, takeout, etc.): Film Production
Near where?: Los Angeles
What sort of industry would you like to disrupt? You may list multiple categories, separated with a comma: Video/Film Production, Recording & Rehearsal Studios, Advertising, Studio Taping


In [57]:
print(categories)

Video/Film Production, Recording & Rehearsal Studios, Advertising, Studio Taping


In [58]:
def yelp_call(headers, url_params):
    """
    This function will use the url_params variable and the headers variable to call the Yelp API,
    and return the data as a JSON 
    This will use the requests module to get from Yelp. 
    What is returned will be modified by our URL parameters.
    This must be called fresh with updated url_params for each call if we want to return more results.
    """
    response = requests.get(url, headers=headers, params=url_params) # our url, header and params should be consistent, atleast with our Yelp data
    data = response.json()
    return data

In [59]:
def parse_data(list_of_data):
    """
    Input data['businesses'] to return a list of tuples,
    with each tuple containing individual business name, address, rating, review count,
    Categories, and business ID
    """
    businesses = []
    for business in list_of_data:
        biz_price = None
        if 'price' not in business.keys():
            biz_price = None
        else:
            biz_price = len(business['price'])
        biz_tuple = (business['name'],
                     business['location']['display_address'],
                     business['location']['city'],
                     business['rating'],
                     business['review_count'],
                     business['coordinates'],
                     biz_price,
                     business['id'],
                     business['categories'])
        businesses.append(biz_tuple)
    return businesses

In [60]:
def call_1000():
    """
    This function will use the information gathered above to call the Yelp API and construct a data frame
    """
    csv_filepath = f'../database/{term}_{location}_database.csv'
    url_params['offset'] = 0
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses']) # list of businesses in tuples
    num = results['total']
    biz_list = []
    #Loop through the API to reach all of the businesses in the call
    while url_params['offset'] < 1000 and len(biz_list) < num:
        for biz in parsed:
            biz_list.append(biz)
        url_params['offset'] += 50
        results = yelp_call(headers, url_params)
        if num >= len(biz_list): # The statements below prevent the function from breaking due to the Yelp API restricting returned calls to 1000
            if 'businesses' not in results:
                break
            else:
                parsed = parse_data(results['businesses']) # list of businesses in tuples
        elif len(biz_list) <= 950:
            continue
        else:
            break
    # Create the data frame from the gathered information
    df = pd.DataFrame(biz_list, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price','Id','Categories'])
    #Save the data frame as a CSV file
    with open(csv_filepath, "a") as f: 
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
    print(f'CSV file written to {csv_filepath}.')
    return df.head(3)

### Run below to write your business data to a csv file.

In [61]:
call_1000()

CSV file written to ../database/Film Production_Atlanta_database.csv.


,Name,Address,City,Rating,Review Count,Coordinates,Price,Id,Categories
0,Make a Film in a Day,"[Atlanta, GA 30310]",Atlanta,5.0,1,"{'latitude': 33.7410049438477, 'longitude': -8...",NaN,1Xfk4fY3jYgctyYYT6e2RA,"[{'alias': 'videofilmproductions', 'title': 'V..."
1,Tyler Perry Studios,"[541 10th St NW, Atlanta, GA 30318]",Atlanta,4.5,6,"{'latitude': 33.7816566, 'longitude': -84.4050...",NaN,IiQ5du4bRMbu-srtdB4UnA,"[{'alias': 'videofilmproductions', 'title': 'V..."
2,Prominence Films,"[Atlanta, GA 30309]",Atlanta,5.0,1,"{'latitude': 33.7985877990723, 'longitude': -8...",NaN,ZSFrOmkul2XQprE3ZZ8pXQ,"[{'alias': 'videofilmproductions', 'title': 'V..."


#### With that file, you can use the functions below to call the Yelp API for the associated business reviews.

In [62]:
term_loc = f'../database/{term}_{location}_database.csv'
biz_data = pd.read_csv(term_loc)

In [63]:
def call_reviews(biz_id): 
    """
    This function loops through the list of business ID's, and call the API on each one.
    Then, it will save this data to a list, and return the list
    """
    list_of_reviews = []
    for biz in biz_id:
        response = requests.get(f'https://api.yelp.com/v3/businesses/{biz}/reviews',headers = headers)
        review_data = response.json()
        list_of_reviews.append(review_data)
    return list_of_reviews 

In [64]:
def call_all_reviews(b_data): 
    """
    This function takes in a data frame, which we can create from our written csv files, 
    and creates a list of the business Id's from it. 
    This will then return that list
    """
    biz_id = []
    for j in b_data['Id']: # iterated through the ID column of business data dataframe and appends each item to a list
        biz_id.append(j)
    list_of_reviews = call_reviews(biz_id)
    return list_of_reviews

In [65]:
def format_reviews(b_data):
    """
    This function takes in the business data frame, and calls the function 'call_all_reviews' 
    to get the list of business Ids.  It will then loop through and create a new list of dictionaries
    with all of the reviews for that company, and that companies Business Id.
    """
    eg = call_all_reviews(b_data)
    list_of_reviews = []
    x = 0
    for i in eg:
        reviews = {}
        for count in list(range(0, (len(i['reviews'])))): # This may be confusing, but it is simply comparing
            reviews[f'Review_{count}'] = i['reviews'][count]['text'] 
        reviews['Id'] = b_data["Id"][x]
        list_of_reviews.append(reviews)
        x+=1
    return list_of_reviews

In [66]:
def reviews_to_csv(b_data): # WE NEED TO FIX THIS
    """
    This function takes in the business data frame, and runs the 'format_reviews' function.
    It then converts the list of dictionaries into the reviews CSV file
    """
    csv_filepath = f'../database/{term}_{location}_reviews.csv'
    formatted_reviews = format_reviews(b_data)
    df = pd.DataFrame(formatted_reviews)
    with open(csv_filepath, "a") as f:
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
    return df

<br><br><br><br><br><br><br><br><br><br><br><br><br>

Below only works if the list of businesses is less than 1000.

In [67]:
reviews_to_csv(biz_data)

,Review_0,Id,Review_1,Review_2
0,Such a great experience! The whole thing was m...,1Xfk4fY3jYgctyYYT6e2RA,NaN,NaN
1,I recently had the opportunity to work as an e...,IiQ5du4bRMbu-srtdB4UnA,I am all for this man and his studio. God bles...,It's an honor to demonstrate to my children th...
2,We hired Prominence Films on a job for a natio...,ZSFrOmkul2XQprE3ZZ8pXQ,NaN,NaN
3,It's great to know the ATLANTA production comp...,z_K-bn1VP3ED3bi1Oj661g,NaN,NaN
4,Great studio with green screen. Was here for a...,Lj6fBJeu1p__HcJU1T4Q9w,Outstanding customer service: A client of mine...,NaN
5,Alex is one of the best videographers i've eve...,D8hmxEIid5W6cr4ASlApMQ,NaN,NaN
6,"One of the nicest, most state of the art produ...",4GA1ZJaBiGQZEGQP4D6AvQ,NaN,NaN
7,Great video production team. We got exactly wh...,I8Qq3abrlCM0tf15WcbDrA,NaN,NaN
8,Excellent company to work with and would recom...,cHRtR8SoOwRYcruW92C_0g,NaN,NaN
9,Want to see something cool? Step inside ECG's...,jG1sRe0ab5nTHRDhF2QUoA,"I've not actually worked with them-- however, ...",After working with some of the premier product...


<br><br><br><br><br><br><br><br><br><br><br><br>